In [1]:
!pwd

/home/nlp/preprocess


In [2]:
import os
import re
from tqdm import tqdm

In [3]:
# DATA PATH

data_path = "/home/nlp/preprocess/datas/"
data_name = "kowiki-20220501.txt"

data = data_path + data_name

# OUTPUT PATH
output_path = "/home/nlp/preprocess/outputs/"
output = output_path + data_name[:-4] + "-preprocessed.txt"

In [4]:
with open(data, "r") as f:
    data = f.read()

In [5]:
print(data[:1000])

<doc id="5" url="https://ko.wikipedia.org/wiki?curid=5" title="지미 카터">
지미 카터

제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대 대통령 (1977년 ~ 1981년)이다.
생애.
어린 시절.
지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.
조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.
정계 입문.
1962년 조지아주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.
대통령 재임.
1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워서, 많은 지지를 받고 제럴드 포드 대통령을 누르고 당선되었다.
카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.
외교 정책.
카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 그러나 1979년, 양국 간의 평화조약이 백악관에서 이루어졌다.
소련과 제2차 전략 무기 제한 협상(SALT II)에 조인했다.
카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으며, 취임 이후 계속해서 도덕정치를 내세웠다.
임기 말, 소련의 아프가니스탄 침공 사건으로 인해 1980년 하계 올림픽에 반공국가들의 보이콧을 하였

In [6]:
lines = data.split("\n")
print(lines[:10])

['<doc id="5" url="https://ko.wikipedia.org/wiki?curid=5" title="지미 카터">', '지미 카터', '', '제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대 대통령 (1977년 ~ 1981년)이다.', '생애.', '어린 시절.', '지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.', '조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.', '정계 입문.', '1962년 조지아주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.']


In [7]:
filtered_lines = []

bracket_text = re.compile(' *\([^)]*\)')  # (), (내용), ...
korean_text = re.compile('[가-힣]')
number_text1 = re.compile('\d+보다 크고 (자연수 )*\d+보다 작은 자연수')
number_text2 = re.compile('\d{3}\([가-힣]+, [A-Z]+, [A-Z0-9]+\)')
number_text3 = re.compile('(\d+, ){5,}')
school_name = re.compile('(\S*학교* ){5,}')
law_text = re.compile('\[시행 \d+\.\d+\.\d+\.\] \[.* \d+\.\d+\.\d+\..*\]')
election_text1 = re.compile('다음은 제\d회 .*선거')
election_text2 = re.compile('이 문서는 제\d회 .*선거')
file_text = re.compile('\[\[([^\[\]]*\|){2,}[^\[\]]*\]\]')  # [[파일:ArafatEconomicForum.jpg|섬네일|왼쪽|2001년 경제 포럼]]
line_text = re.compile('━{2,}┳━{2,}')
event_text = re.compile('\d+년 기준.*에서 열리는.*는 총')
none_text = re.compile('[]')
usnavy_text = re.compile('\d+\. .*\[[A-Z]+-[0-9]+\] [\d진]')
broadcast_text = re.compile('방송일시* : \d+월 \d+일\([가-힣]\).*\d+시')
year_text = re.compile('\d{4}\. \d{1,2} ')
special_text = re.compile('[-!/\\#＃&＆*＊@＠§※☆★○●◎◇◆□■△▲▽▼→←↑↓↔〓◁◀▷▶♤♠♡♥♧♣⊙◈▣☞•]')
space_text = re.compile(' {2,}')
date_text1 = re.compile(' *\d{8}')
date_text2 = re.compile(' *\d{4} \d{2} ')
date_text3 = re.compile(' *\d{4}\. \d{1,2}\. \d{1,2}')
date_text4 = re.compile(' *\d{4}-\d{1,2}-\d{1,2}')
date_text5 = re.compile(' *\d* \[\d{4}\. *\d{1,2}\. *\d{1,2}')
gregory_text = re.compile('\d+월 \d+일은 그레고리력으로.*해당한다')

link_text = re.compile('\[\[[가-힣a-zA-Z0-9 ]+\|([가-힣a-zA-Z0-9 ]+)\]\]')

for line in tqdm(lines):
    # remove contents with brackets
    line = line.strip()
    line = bracket_text.sub("", line)
    
    # write <|endoftext|> when the line is document info&link or sentence is too short.
    if line.startswith("<doc id") or line.startswith("</doc>") or line.startswith("&lt;"):
        line = "<|endoftext|>"
    elif len(line) <= 30:
        line = "<|endoftext|>"
    elif line.startswith("이 문서는"):
        line = "<|endoftext|>"
    elif line.endswith("목록을 나열한다."):
        line = "<|endoftext|>"
    elif line.startswith("&lt;") or line.startswith(" &lt;"):
        line = "<|endoftext|>"
    elif special_text.match(line) != None:
        line = "<|endoftext|>"
    elif korean_text.search(line) == None:
        line = "<|endoftext|>"
    elif number_text1.search(line) != None:
        line = "<|endoftext|>"
    elif number_text2.search(line) != None:
        line = "<|endoftext|>"
    elif number_text3.search(line) != None:
        line = "<|endoftext|>"
    elif school_name.search(line) != None:
        line = "<|endoftext|>"
    elif law_text.search(line) != None:
        line = "<|endoftext|>"
    elif election_text1.search(line) != None or election_text2.search(line) != None:
        line = "<|endoftext|>"
    elif file_text.search(line) != None:
        line = "<|endoftext|>"
    elif line_text.search(line) != None:
        line = "<|endoftext|>"
    elif event_text.search(line) != None:
        line = "<|endoftext|>"
    elif none_text.search(line) != None:
        line = "<|endoftext|>"
    elif usnavy_text.search(line) != None:
        line = "<|endoftext|>"
    elif broadcast_text.search(line) != None:
        line = "<|endoftext|>"
    elif year_text.match(line) != None:
        line = "<|endoftext|>"
    elif date_text1.match(line) != None:
        line = "<|endoftext|>"
    elif date_text2.match(line) != None:
        line = "<|endoftext|>"
    elif date_text3.match(line) != None:
        line = "<|endoftext|>"
    elif date_text4.match(line) != None:
        line = "<|endoftext|>"
    elif date_text5.match(line) != None:
        line = "<|endoftext|>"
    elif gregory_text.match(line) != None:
        line = "<|endoftext|>"
    
    # to prevent write <|endoftext|> twice
    if line == "<|endoftext|>":
        if len(filtered_lines) == 0:
            continue
        if filtered_lines[-1] == "<|endoftext|>":
            continue
        filtered_lines.append(line)
        continue
    
    
    link_check = link_text.search(line)
    if link_check != None:
        line = line.replace(link_check.group(0), link_check.group(1))
    
    # change the special characters to normal charcters
    # ㅁ
    line = line.replace("＃", "#")
    line = line.replace("＆", "&")
    line = line.replace("＊", "*")
    line = line.replace("＠", "@")
    # ㄴ
    line = line.replace("＂", "\"")
    line = line.replace("（", "(")
    line = line.replace("）", ")")
    line = line.replace("［", "[")
    line = line.replace("］", "]")
    line = line.replace("｛", "{")
    line = line.replace("｝", "}")
    line = line.replace("‘", "'")
    line = line.replace("’", "'")
    line = line.replace("“", "\"")
    line = line.replace("”", "\"")
    line = line.replace("〔", "[")
    line = line.replace("〕", "]")
    line = line.replace("〈", "<")
    line = line.replace("〉", ">")
    # ㄹ
    line = line.replace("％", "%")
    # ㄱ
    line = line.replace("　", " ")
    line = line.replace("！", "!")
    line = line.replace("＇", "'")
    line = line.replace("，", ",")
    line = line.replace("．", ".")
    line = line.replace("／", "/")
    line = line.replace("：", ":")
    line = line.replace("；", ";")
    line = line.replace("？", "?")
    line = line.replace("、", ",")
    line = line.replace("∼", "~")
    line = line.replace("´", "'")
    line = line.replace("～", "~")
    # ㄷ
    line = line.replace("＋", "+")
    line = line.replace("－", "-")
    line = line.replace("＜", "<")
    line = line.replace("＝", "=")
    line = line.replace("＞", ">")
    
    # edit link text not parsed well
    line = line.replace("[[", "")
    line = line.replace("]]", "")
    
    # edit special characters
    line = line.replace("&lt;", "<")
    line = line.replace("&gt;", ">")
    line = line.replace("&amp;", "&")
    line = line.replace("&quot;", "\"")

    # prevent more than 2 spaces
    line = space_text.sub(" ", line)

    filtered_lines.append(line)

100%|██████████| 10198884/10198884 [02:43<00:00, 62423.28it/s]


In [8]:
filtered_data = "\n".join(filtered_lines)
filtered_data = filtered_data.replace("\n<|endoftext|>", "<|endoftext|>")

In [9]:
with open(output, "w") as f:
    f.write(filtered_data)